# Annotate Latin Inscriptions with Gemini API

This notebook provides a complete workflow for:
1. Downloading epigraphic data from EDH (Epigraphic Database Heidelberg)
2. Annotating inscriptions using Gemini Flash 2.5 API
3. Validating annotation quality
4. Saving results and committing to git

## Prerequisites

1. **Google AI API Key**: Get one at https://aistudio.google.com/app/apikey
2. **Installed packages**: Run the setup cell below

## Cost Estimate

- Gemini Flash 2.5: ~$0.20-0.50 USD per 1000 inscriptions
- Processing time: ~20-40 minutes per 1000 inscriptions (with 1s delay)

In [8]:
from google.colab import userdata
key = userdata.get('GOOGLE_API_KEY')

## 1. Setup and Installation

In [2]:
# Install required packages
!pip install -q google-generativeai pandas requests

# Install latinepi if not already installed
# import os
# if not os.path.exists('latinepi'):
#     print("Installing latinepi package...")
#     !pip install -e . -q
# else:
#     print("latinepi already available")

In [3]:
# Imports
import json
import time
import re
from pathlib import Path
from datetime import datetime
from typing import List, Dict, Optional
import pandas as pd
from collections import Counter, defaultdict

try:
    import google.generativeai as genai
    print("✅ Google Generative AI imported successfully")
except ImportError:
    print("❌ Please run the installation cell above")
    raise

✅ Google Generative AI imported successfully


In [4]:
!wget https://raw.githubusercontent.com/shawngraham/latinepi/refs/heads/main/gemini_annotation_prompt.md

--2025-11-18 23:05:10--  https://raw.githubusercontent.com/shawngraham/latinepi/refs/heads/main/gemini_annotation_prompt.md
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.110.133, 185.199.111.133, 185.199.108.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.110.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 16333 (16K) [text/plain]
Saving to: ‘gemini_annotation_prompt.md’

gemini_annotation_p 100%[===================>]  15.95K  --.-KB/s    in 0.001s  

2025-11-18 23:05:11 (16.3 MB/s) - ‘gemini_annotation_prompt.md’ saved [16333/16333]



## 2. Configuration

In [22]:
# ========================================
# CONFIGURATION - Adjust these settings
# ========================================

# Google AI API Key
GOOGLE_AI_API_KEY = key #os.environ.get("GOOGLE_AI_API_KEY", "")

if not GOOGLE_AI_API_KEY:
    print("⚠️  No API key found in environment.")
    print("   Option 1: Set environment variable: export GOOGLE_AI_API_KEY='your-key'")
    print("   Option 2: Enter it directly in the next cell")
else:
    print(f"✅ API key loaded (first 10 chars): {GOOGLE_AI_API_KEY[:10]}...")

# Gemini model to use
#GEMINI_MODEL = "gemini-2.0-flash-exp"  # Fast and cost-effective
GEMINI_MODEL = "gemini-flash-lite-latest"
# Alternative: "gemini-1.5-pro" for higher quality (more expensive)

# Output paths
OUTPUT_DIR = Path("assets")
OUTPUT_DIR.mkdir(exist_ok=True)

ANNOTATIONS_FILE = OUTPUT_DIR / "gemini_annotations.jsonl"
CHECKPOINT_FILE = OUTPUT_DIR / "gemini_annotations.jsonl.tmp"

# Processing parameters
SAVE_CHECKPOINT_EVERY = 50  # Save progress every N inscriptions
API_DELAY = 1.0  # Seconds between API calls (rate limiting)
MAX_RETRIES = 3  # Retry failed API calls

print("\n📋 Configuration loaded:")
print(f"   Model: {GEMINI_MODEL}")
print(f"   Output: {ANNOTATIONS_FILE}")
print(f"   Checkpoint every: {SAVE_CHECKPOINT_EVERY} inscriptions")
print(f"   API delay: {API_DELAY}s")

✅ API key loaded (first 10 chars): ...

📋 Configuration loaded:
   Model: gemini-flash-lite-latest
   Output: assets/gemini_annotations.jsonl
   Checkpoint every: 50 inscriptions
   API delay: 1.0s


In [ ]:
# If you need to set API key directly (not from environment):
# GOOGLE_AI_API_KEY = "paste-your-key-here"
# genai.configure(api_key=GOOGLE_AI_API_KEY)

In [17]:
def load_inscriptions_from_csv(csv_path: Path,
                                text_column: str = "clean_text",
                                id_column: str = "id") -> List[Dict]:
    """
    Load inscriptions from a CSV file with id and clean_text columns.

    Args:
        csv_path: Path to CSV file
        text_column: Name of column containing cleaned text (default: "clean_text")
        id_column: Name of column containing inscription IDs (default: "id")

    Returns:
        List of dicts with keys: id, text, transcription
    """
    df = pd.read_csv(csv_path)

    inscriptions = []

    for idx, row in df.iterrows():
        # Get raw values
        inscription_id = row.get(id_column, f"row_{idx}")
        clean_text = row.get(text_column, "")

        # Handle NaN values
        if pd.isna(clean_text):
            clean_text = ""

        # Only include if we have text
        if clean_text and len(str(clean_text).strip()) > 2:
            inscriptions.append({
                "id": str(inscription_id),
                "text": "",  # No diplomatic text in your CSV
                "transcription": str(clean_text)  # This is what gets annotated
            })

    return inscriptions


# Usage:
CSV_FILE = Path("cleaned_inscriptions.csv")
inscriptions = load_inscriptions_from_csv(
    CSV_FILE,
    text_column="clean_text",
    id_column="id"
)

print(f"✅ Loaded {len(inscriptions)} inscriptions from CSV")

✅ Loaded 2000 inscriptions from CSV


In [13]:
inscriptions

[]

## 3. Download Inscription Data from EDH

We'll use the `latinepi` package to search and download inscriptions from the Epigraphic Database Heidelberg.

In [ ]:
# EDH Search Parameters
# Adjust these to get different inscription sets

SEARCH_PARAMS = {
    "year_from": 1,      # 1 CE
    "year_to": 100,      # 100 CE (1st century)
    "limit": 2000,       # Number of inscriptions to download
}

DOWNLOAD_DIR = Path("edh_downloads/batch_" + datetime.now().strftime("%Y%m%d_%H%M%S"))

print(f"🔍 Search parameters:")
for key, value in SEARCH_PARAMS.items():
    print(f"   {key}: {value}")
print(f"\n📂 Will download to: {DOWNLOAD_DIR}")

In [ ]:
# Download inscriptions from EDH using latinepi command-line tool
import subprocess

DOWNLOAD_DIR.mkdir(parents=True, exist_ok=True)

cmd = [
    "latinepi",
    "--search-edh",
    "--search-year-from", str(SEARCH_PARAMS["year_from"]),
    "--search-year-to", str(SEARCH_PARAMS["year_to"]),
    "--search-limit", str(SEARCH_PARAMS["limit"]),
    "--download-dir", str(DOWNLOAD_DIR),
]

print(f"⏳ Downloading {SEARCH_PARAMS['limit']} inscriptions from EDH...")
print(f"   Command: {' '.join(cmd)}")
print("\n" + "="*60)

try:
    result = subprocess.run(cmd, capture_output=True, text=True, timeout=300)
    print(result.stdout)
    if result.stderr:
        print("Errors/Warnings:")
        print(result.stderr)

    # Count downloaded files
    json_files = list(DOWNLOAD_DIR.glob("*.json"))
    print("\n" + "="*60)
    print(f"✅ Downloaded {len(json_files)} inscription files")

except subprocess.TimeoutExpired:
    print("⚠️  Download timed out after 5 minutes")
except Exception as e:
    print(f"❌ Error during download: {e}")

## 4. Load and Prepare Inscription Data

In [19]:
def clean_leiden_text(text):
    """
    Clean Leiden convention markup from inscription text.
    Converts diplomatic transcription to plain Latin text.
    """
    if pd.isna(text) or text == "":
        return ""

    text = str(text)

    # Remove lost text markers
    text = re.sub(r"\[\s*-+\??\s*\]", "", text)
    text = re.sub(r"-+\]", "", text)
    text = re.sub(r"\[-+", "", text)

    # Replace line breaks with spaces
    text = text.replace("/", " ")

    # Remove Leiden markup
    text = text.replace("(", "").replace(")", "")  # Abbreviations
    text = text.replace("[", "").replace("]", "")  # Restorations
    text = text.replace("?", "")  # Uncertain readings

    # Normalize whitespace
    text = re.sub(r"\s+", " ", text).strip()

    return text


def load_inscriptions_from_json_dir(json_dir: Path) -> List[Dict]:
    """
    Load and clean inscriptions from a directory of JSON files.

    Returns:
        List of dicts with keys: id, text, transcription
    """
    json_files = list(json_dir.glob("*.json"))

    inscriptions = []

    for json_file in json_files:
        try:
            with open(json_file, 'r', encoding='utf-8') as f:
                data = json.load(f)

            # Get diplomatic text and transcription
            raw_text = data.get("diplomatic_text", "")
            transcription = data.get("transcription", "")

            # Clean the transcription
            cleaned = clean_leiden_text(transcription)

            # Only include if we have text
            if cleaned and len(cleaned.strip()) > 2:
                inscriptions.append({
                    "id": data.get("id", json_file.stem),
                    "text": raw_text,
                    "transcription": cleaned
                })

        except Exception as e:
            print(f"⚠️  Error loading {json_file.name}: {e}")
            continue

    return inscriptions


# Load inscriptions
#print(f"📚 Loading inscriptions from {DOWNLOAD_DIR}...")

CSV_FILE = Path("cleaned_inscriptions.csv")
inscriptions = load_inscriptions_from_csv(
    CSV_FILE,
    text_column="clean_text",
    id_column="id"
)
#inscriptions = load_inscriptions_from_json_dir(DOWNLOAD_DIR)

print(f"\n✅ Loaded {len(inscriptions)} inscriptions with valid text")

# Show sample
if inscriptions:
    print("\n📋 Sample inscription:")
    sample = inscriptions[0]
    print(f"   ID: {sample['id']}")
    print(f"   Text: {sample['transcription'][:80]}...")
    print(f"   Length: {len(sample['transcription'])} characters")


✅ Loaded 2000 inscriptions with valid text

📋 Sample inscription:
   ID: HD000052
   Text: Pro salute et reditu et victorias! domini nostri Imperatoris Caesaris Marci Aure...
   Length: 277 characters


## 5. Load Annotation Prompt

In [20]:
# Load the annotation prompt
PROMPT_FILE = Path("gemini_annotation_prompt.md")

if not PROMPT_FILE.exists():
    print(f"❌ Prompt file not found: {PROMPT_FILE}")
    print("   Make sure you're running this notebook from the repository root")
else:
    with open(PROMPT_FILE, 'r', encoding='utf-8') as f:
        ANNOTATION_PROMPT = f.read()

    print(f"✅ Loaded annotation prompt from {PROMPT_FILE}")
    print(f"   Prompt length: {len(ANNOTATION_PROMPT)} characters")
    print(f"   Estimated tokens: ~{len(ANNOTATION_PROMPT.split()) * 1.3:.0f}")

✅ Loaded annotation prompt from gemini_annotation_prompt.md
   Prompt length: 16238 characters
   Estimated tokens: ~3062


## 6. Initialize Gemini API

In [23]:
# Configure Gemini API
if not GOOGLE_AI_API_KEY:
    raise ValueError(
        "No API key set! Either:\n"
        "1. Set environment variable: export GOOGLE_AI_API_KEY='your-key'\n"
        "2. Or set it directly in the configuration cell above"
    )

genai.configure(api_key=GOOGLE_AI_API_KEY)

# Initialize the model
model = genai.GenerativeModel(
    model_name=GEMINI_MODEL,
    generation_config={
        "temperature": 0.1,  # Low temperature for deterministic output
        "top_p": 0.95,
        "top_k": 40,
        "max_output_tokens": 2048,
    }
)

print(f"✅ Gemini API configured")
print(f"   Model: {GEMINI_MODEL}")
print(f"   Temperature: 0.1 (deterministic)")
print(f"\n🧪 Testing API connection...")

try:
    test_response = model.generate_content("Say 'API connection successful' in exactly those words.")
    print(f"   Response: {test_response.text.strip()}")
    print("   ✅ API is working!")
except Exception as e:
    print(f"   ❌ API test failed: {e}")
    raise

✅ Gemini API configured
   Model: gemini-flash-lite-latest
   Temperature: 0.1 (deterministic)

🧪 Testing API connection...


ERROR:tornado.access:503 POST /v1beta/models/gemini-flash-lite-latest:generateContent?%24alt=json%3Benum-encoding%3Dint (::1) 2982.65ms


   Response: API connection successful
   ✅ API is working!


## 7. Annotation Functions

In [24]:
def annotate_single_inscription(inscription: Dict, prompt: str, model) -> Optional[Dict]:
    """
    Annotate a single inscription using Gemini API.

    Args:
        inscription: Dict with keys: id, text, transcription
        prompt: The annotation prompt (system instructions)
        model: Gemini model instance

    Returns:
        Dict with annotations added, or None if failed
    """
    # Prepare input JSON
    input_json = json.dumps({
        "id": inscription.get("id", ""),
        "text": inscription.get("text", ""),
        "transcription": inscription.get("transcription", "")
    }, ensure_ascii=False, indent=2)

    # Construct full prompt
    full_prompt = f"""{prompt}

---

Please annotate the following inscription:

{input_json}

Return ONLY the JSON object with annotations added. No other text.
"""

    # Try API call with retries
    for attempt in range(MAX_RETRIES):
        try:
            response = model.generate_content(full_prompt)
            response_text = response.text.strip()

            # Clean up response
            if response_text.startswith("```json"):
                response_text = response_text[7:]
            if response_text.startswith("```"):
                response_text = response_text[3:]
            if response_text.endswith("```"):
                response_text = response_text[:-3]
            response_text = response_text.strip()

            # Parse JSON
            result = json.loads(response_text)

            # Validate structure
            if "annotations" not in result:
                print(f"⚠️  Missing 'annotations' field for {inscription.get('id')}")
                return None

            if not isinstance(result["annotations"], list):
                print(f"⚠️  Invalid annotations format for {inscription.get('id')}")
                return None

            return result

        except json.JSONDecodeError as e:
            if attempt < MAX_RETRIES - 1:
                time.sleep(2 ** attempt)  # Exponential backoff
                continue
            print(f"❌ JSON parse error for {inscription.get('id')} after {MAX_RETRIES} attempts")
            return None

        except Exception as e:
            if attempt < MAX_RETRIES - 1:
                time.sleep(2 ** attempt)
                continue
            print(f"❌ API error for {inscription.get('id')}: {e}")
            return None

    return None


def save_checkpoint(results: List[Dict], checkpoint_file: Path):
    """Save intermediate results to checkpoint file."""
    with open(checkpoint_file, 'w', encoding='utf-8') as f:
        for result in results:
            f.write(json.dumps(result, ensure_ascii=False) + '\n')


def save_final(results: List[Dict], output_file: Path, checkpoint_file: Path):
    """Save final results and clean up checkpoint."""
    with open(output_file, 'w', encoding='utf-8') as f:
        for result in results:
            # Remove any error markers
            clean_result = {k: v for k, v in result.items() if k != "_error"}
            f.write(json.dumps(clean_result, ensure_ascii=False) + '\n')

    # Clean up checkpoint
    if checkpoint_file.exists():
        checkpoint_file.unlink()

print("✅ Annotation functions defined")

✅ Annotation functions defined


## 8. Run Batch Annotation

⚠️ **Important**: This cell will make API calls and may take 30-60 minutes for 2000 inscriptions.

Progress is saved every 50 inscriptions, so you can interrupt and resume if needed.

In [25]:
# Optional: Limit number of inscriptions for testing
TEST_LIMIT = None  # Set to e.g. 10 for testing, None for all

if TEST_LIMIT:
    inscriptions_to_process = inscriptions[:TEST_LIMIT]
    print(f"⚠️  TEST MODE: Processing only {TEST_LIMIT} inscriptions")
else:
    inscriptions_to_process = inscriptions
    print(f"📝 Processing all {len(inscriptions)} inscriptions")

# Check for existing checkpoint
completed = []
resume_from = 0

if CHECKPOINT_FILE.exists():
    print(f"\n📂 Found checkpoint file: {CHECKPOINT_FILE}")
    with open(CHECKPOINT_FILE, 'r', encoding='utf-8') as f:
        completed = [json.loads(line) for line in f]
    resume_from = len(completed)
    print(f"   Resuming from index {resume_from}")

# Statistics
stats = {
    'start_time': datetime.now(),
    'total': len(inscriptions_to_process),
    'successful': len(completed),
    'failed': 0,
    'total_entities': sum(len(r.get('annotations', [])) for r in completed)
}

print(f"\n🚀 Starting annotation...")
print(f"   Total: {stats['total']}")
print(f"   Already completed: {len(completed)}")
print(f"   Remaining: {stats['total'] - resume_from}")
print(f"   Save checkpoint every: {SAVE_CHECKPOINT_EVERY}")
print(f"   API delay: {API_DELAY}s")
print("\n" + "="*70)

# Process inscriptions
for i, inscription in enumerate(inscriptions_to_process[resume_from:], start=resume_from):
    inscription_id = inscription.get('id', f'index_{i}')

    # Progress indicator
    progress_pct = (i + 1) / stats['total'] * 100
    print(f"[{i+1}/{stats['total']} ({progress_pct:.1f}%)] {inscription_id}... ", end="", flush=True)

    # Annotate
    result = annotate_single_inscription(inscription, ANNOTATION_PROMPT, model)

    if result:
        completed.append(result)
        num_entities = len(result.get('annotations', []))
        stats['successful'] += 1
        stats['total_entities'] += num_entities
        print(f"✓ ({num_entities} entities)")
    else:
        # Save failed case with error marker
        completed.append({
            **inscription,
            "annotations": [],
            "_error": True
        })
        stats['failed'] += 1
        print("✗ (failed)")

    # Save checkpoint
    if (i + 1) % SAVE_CHECKPOINT_EVERY == 0:
        save_checkpoint(completed, CHECKPOINT_FILE)
        elapsed = (datetime.now() - stats['start_time']).total_seconds()
        rate = (i + 1) / elapsed
        remaining = (stats['total'] - i - 1) / rate if rate > 0 else 0
        print(f"   💾 Checkpoint saved ({len(completed)} completed, ~{remaining/60:.1f} min remaining)")

    # Rate limiting
    if i < stats['total'] - 1:
        time.sleep(API_DELAY)

# Save final results
print("\n" + "="*70)
print("💾 Saving final results...")
save_final(completed, ANNOTATIONS_FILE, CHECKPOINT_FILE)

# Final statistics
elapsed = (datetime.now() - stats['start_time']).total_seconds()
avg_entities = stats['total_entities'] / max(1, stats['successful'])

print("\n" + "="*70)
print("✅ ANNOTATION COMPLETE")
print("="*70)
print(f"Total inscriptions: {stats['total']}")
print(f"Successful: {stats['successful']}")
print(f"Failed: {stats['failed']}")
print(f"Success rate: {stats['successful']/stats['total']*100:.1f}%")
print(f"\nTotal entities annotated: {stats['total_entities']}")
print(f"Avg entities per inscription: {avg_entities:.1f}")
print(f"\nTime elapsed: {elapsed/60:.1f} minutes")
print(f"Rate: {stats['total']/elapsed*60:.1f} inscriptions/minute")
print(f"\nOutput saved to: {ANNOTATIONS_FILE}")
print("="*70)

📝 Processing all 2000 inscriptions

🚀 Starting annotation...
   Total: 2000
   Already completed: 0
   Remaining: 2000
   Save checkpoint every: 50
   API delay: 1.0s

[1/2000 (0.1%)] HD000052... 

ERROR:tornado.access:503 POST /v1beta/models/gemini-flash-lite-latest:generateContent?%24alt=json%3Benum-encoding%3Dint (::1) 1391.02ms
ERROR:tornado.access:503 POST /v1beta/models/gemini-flash-lite-latest:generateContent?%24alt=json%3Benum-encoding%3Dint (::1) 1111.63ms
ERROR:tornado.access:503 POST /v1beta/models/gemini-flash-lite-latest:generateContent?%24alt=json%3Benum-encoding%3Dint (::1) 9651.87ms
ERROR:tornado.access:503 POST /v1beta/models/gemini-flash-lite-latest:generateContent?%24alt=json%3Benum-encoding%3Dint (::1) 1466.61ms
ERROR:tornado.access:503 POST /v1beta/models/gemini-flash-lite-latest:generateContent?%24alt=json%3Benum-encoding%3Dint (::1) 1241.44ms


KeyboardInterrupt: 

## 9. Validate Annotations

Run quality checks on the annotated data.

In [ ]:
# Quick validation
print("🔍 Running validation checks...\n")

# Load results
with open(ANNOTATIONS_FILE, 'r', encoding='utf-8') as f:
    results = [json.loads(line) for line in f]

# Count entity types
entity_counts = Counter()
errors = []
overlapping = []

for record in results:
    text = record.get('transcription', '')
    annotations = record.get('annotations', [])

    if not annotations:
        continue

    # Count entity types
    for ann in annotations:
        if len(ann) == 3:
            start, end, label = ann
            entity_counts[label] += 1

            # Check bounds
            if start < 0 or end > len(text) or start >= end:
                errors.append({
                    'id': record.get('id'),
                    'error': f'Invalid bounds [{start}:{end}] for text length {len(text)}'
                })

    # Check for overlaps
    sorted_anns = sorted(annotations, key=lambda x: x[0])
    for i in range(len(sorted_anns) - 1):
        if sorted_anns[i][1] > sorted_anns[i+1][0]:  # end[i] > start[i+1]
            overlapping.append({
                'id': record.get('id'),
                'overlap': f'{sorted_anns[i]} overlaps with {sorted_anns[i+1]}'
            })

# Print results
print("📊 Entity Type Distribution:")
print(f"{'Entity Type':<30} {'Count':>8}")
print("-" * 40)
for label, count in entity_counts.most_common():
    print(f"{label:<30} {count:8d}")

print(f"\n🔍 Quality Checks:")
print(f"   Total records: {len(results)}")
print(f"   Records with annotations: {sum(1 for r in results if r.get('annotations'))}")
print(f"   Total entities: {sum(entity_counts.values())}")

if errors:
    print(f"\n❌ Found {len(errors)} errors:")
    for err in errors[:5]:
        print(f"   {err['id']}: {err['error']}")
    if len(errors) > 5:
        print(f"   ... and {len(errors) - 5} more")
else:
    print(f"\n✅ No boundary errors found")

if overlapping:
    print(f"\n⚠️  Found {len(overlapping)} overlapping spans:")
    for ovr in overlapping[:5]:
        print(f"   {ovr['id']}: {ovr['overlap']}")
    if len(overlapping) > 5:
        print(f"   ... and {len(overlapping) - 5} more")
else:
    print(f"\n✅ No overlapping spans found")

# Overall quality
error_rate = len(errors) / max(1, len(results))
overlap_rate = len(overlapping) / max(1, len(results))

if error_rate == 0 and overlap_rate == 0:
    quality = "🌟 EXCELLENT"
elif error_rate < 0.01 and overlap_rate < 0.05:
    quality = "✅ GOOD"
elif error_rate < 0.05 and overlap_rate < 0.10:
    quality = "⚠️  FAIR"
else:
    quality = "❌ NEEDS REVIEW"

print(f"\n🎯 Overall Quality: {quality}")
print(f"   Error rate: {error_rate:.2%}")
print(f"   Overlap rate: {overlap_rate:.2%}")

## 10. Show Sample Annotations

In [ ]:
# Display a few sample annotations
import random

samples = [r for r in results if r.get('annotations')]
if samples:
    print("📋 Sample Annotated Inscriptions:\n")

    for i, record in enumerate(random.sample(samples, min(3, len(samples))), 1):
        print("=" * 70)
        print(f"Sample {i}")
        print("=" * 70)
        print(f"ID: {record.get('id', 'N/A')}")
        print(f"Text: {record['transcription']}")
        print(f"\nEntities ({len(record['annotations'])}):")

        for start, end, label in record['annotations'][:15]:  # Max 15 entities
            entity_text = record['transcription'][start:end]
            print(f"  [{start:3d}:{end:3d}] {label:25s} = '{entity_text}'")

        if len(record['annotations']) > 15:
            print(f"  ... and {len(record['annotations']) - 15} more")
        print()
else:
    print("No annotated samples to display")

## 11. Commit and Push Results

In [ ]:
import subprocess
from datetime import datetime

# Generate commit message
timestamp = datetime.now().strftime("%Y-%m-%d %H:%M")
commit_message = f"""Add Gemini-annotated inscriptions ({timestamp})

Annotated {stats['successful']} inscriptions using {GEMINI_MODEL}
- Total entities: {stats['total_entities']}
- Avg entities per inscription: {avg_entities:.1f}
- Success rate: {stats['successful']/stats['total']*100:.1f}%
- Search params: {SEARCH_PARAMS['year_from']}-{SEARCH_PARAMS['year_to']} CE

Files:
- {ANNOTATIONS_FILE}
- {DOWNLOAD_DIR}/ (raw EDH data)
"""

print("📝 Commit message:")
print("=" * 70)
print(commit_message)
print("=" * 70)
print()

# Ask for confirmation
confirm = input("Commit and push these files? (yes/no): ").strip().lower()

if confirm == 'yes':
    try:
        # Check git status
        print("\n📊 Git status:")
        result = subprocess.run(['git', 'status', '--short'], capture_output=True, text=True)
        print(result.stdout)

        # Add files
        print("\n📦 Adding files to git...")
        subprocess.run(['git', 'add', str(ANNOTATIONS_FILE)], check=True)
        subprocess.run(['git', 'add', str(DOWNLOAD_DIR)], check=True)
        print("   ✅ Files staged")

        # Commit
        print("\n💾 Creating commit...")
        subprocess.run(['git', 'commit', '-m', commit_message], check=True)
        print("   ✅ Commit created")

        # Get current branch
        branch_result = subprocess.run(
            ['git', 'rev-parse', '--abbrev-ref', 'HEAD'],
            capture_output=True,
            text=True,
            check=True
        )
        current_branch = branch_result.stdout.strip()

        # Push
        print(f"\n🚀 Pushing to origin/{current_branch}...")
        subprocess.run(['git', 'push', 'origin', current_branch], check=True)
        print("   ✅ Pushed successfully")

        print("\n✅ All done!")

    except subprocess.CalledProcessError as e:
        print(f"\n❌ Git operation failed: {e}")
        print("   You may need to commit and push manually")

else:
    print("\nℹ️  Skipping git operations")
    print(f"   To commit later, run:")
    print(f"   git add {ANNOTATIONS_FILE} {DOWNLOAD_DIR}")
    print(f"   git commit -m 'Add Gemini annotations'")
    print(f"   git push")

## 12. Summary and Next Steps

In [ ]:
print("="*70)
print("🎉 WORKFLOW COMPLETE")
print("="*70)
print(f"\n📊 Results:")
print(f"   Inscriptions downloaded: {len(json_files)}")
print(f"   Inscriptions annotated: {stats['successful']}")
print(f"   Total entities: {stats['total_entities']}")
print(f"   Output file: {ANNOTATIONS_FILE}")

print(f"\n🔬 Next Steps:")
print(f"\n1. Review annotation quality:")
print(f"   python validate_annotations.py {ANNOTATIONS_FILE}")

print(f"\n2. Use in your training notebook:")
print(f"   INPUT_FILE = '{ANNOTATIONS_FILE}'")
print(f"   partition_data(INPUT_FILE, CLEAN_OUTPUT_FILE, FIX_OUTPUT_FILE)")

print(f"\n3. Compare with synthetic data:")
print(f"   - Train model A: synthetic data only (463 examples)")
print(f"   - Train model B: real data only ({stats['successful']} examples)")
print(f"   - Train model C: combined")
print(f"   - Evaluate all three on held-out real inscriptions")

print(f"\n4. If quality is good, annotate more:")
print(f"   - Adjust SEARCH_PARAMS in cell 3")
print(f"   - Download different time periods/regions")
print(f"   - Aim for 5000+ total inscriptions")

print("\n" + "="*70)